# Введение в MapReduce модель на Python


In [27]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [28]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [29]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [30]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=3, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=1, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [31]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [32]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (3, User(id=3, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (1, User(id=1, age=33, social_contacts=800, gender='female'))]

In [33]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [34]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=3, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=1, age=33, social_contacts=800, gender='female'))]

In [35]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [36]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=3, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=1, age=33, social_contacts=800, gender='female')])]

In [37]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [38]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [39]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.985702460565132),
 (1, 1.985702460565132),
 (2, 1.985702460565132),
 (3, 1.985702460565132),
 (4, 1.985702460565132)]

## Inverted index 

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('it', 18)]),
 (1, [('banana', 2), ('is', 18), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.045368890548610485),
   (None, 0.08693582603366556),
   (None, 0.09416228813503458),
   (None, 0.12344953941819425),
   (None, 0.13449402153780943),
   (None, 0.15165248483695193),
   (None, 0.18080676495180037),
   (None, 0.18765799127233385),
   (None, 0.2013944865011813),
   (None, 0.3113658393599237),
   (None, 0.34836704206406033),
   (None, 0.35610647308415666),
   (None, 0.4349539187571103),
   (None, 0.47093242135181435),
   (None, 0.48677143136978207)]),
 (1,
  [(None, 0.5366529970380596),
   (None, 0.559155245153181),
   (None, 0.5979797558659793),
   (None, 0.6251399237171311),
   (None, 0.6692382266848474),
   (None, 0.6742702462433516),
   (None, 0.6972263573494986),
   (None, 0.7051452287100761),
   (None, 0.788118425821032),
   (None, 0.792086883824465),
   (None, 0.8289911533645138),
   (None, 0.8414719731142087),
   (None, 0.8474084163308054),
   (None, 0.8752874394040976),
   (None, 0.9291944298106183)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:


list1 = [1, 2, 2, 2, 2]
list2 = [1, 'a', 'bb', 'bf4']
list3 = [1, 'x', 'y', 12]

def RECORDREADER():
    yield(0, list1)

def MAP(_, elem):
    elems = []
    for el in elem:
        if type(el) == int:
            elems.append(el)
    yield(0, elems)

def REDUCE(_, elem):
    maxn = max(elem[0])
    yield(maxn)
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[2]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:

list1 = [1, 2, 2, 2, 5]
list2 = [1, 'a', 'bb', 'bf4']
list3 = [1, 'x', 'y', 12]

def RECORDREADER():
    yield(0, list1)

def MAP(_, elem):
    elems = []
    for el in elem:
        if type(el) == int:
            elems.append(el)
    yield(len(elems), elems)

def REDUCE(n, elem):
    S = sum(elem[0])
    #for i in elem[0]:
    #    S += i
    yield(S/n)
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[2.4]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
input_collection = [[0, 2, 1], [1, 3, 1], [2, 1, 0]]
def RECORDREADER():
      return [(u[0], u) for u in input_collection]
def MAP(idn, el):
    yield(idn, el)
def flatten(nested_iterable):
      for iterable in nested_iterable:
        for element in iterable:
              yield element
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
        t[k2][0] = sorted(t[k2][0])
    return t.items()  
    #return [(t_id, sorted(ti)) for (t_id, ti) in enumerate(t)]
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(0, [[0, 1, 2]]), (1, [[1, 1, 3]]), (2, [[0, 1, 2]])]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
input_collection = [[0, 2, 2], [1, 3, 3], [2, 0, 0]]
def RECORDREADER():
      return [(u[0], u) for u in input_collection]
def MAP(idn, el):
    yield(idn, el)
def flatten(nested_iterable):
      for iterable in nested_iterable:
        for element in iterable:
              yield element
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
        t[k2][0] = list(set(t[k2][0]))
    return t.items()  
    #return [(t_id, sorted(ti)) for (t_id, ti) in enumerate(t)]
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(0, [[0, 2]]), (1, [[1, 3]]), (2, [[0, 2]])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [40]:
#Select * from users where gender="female" and age > 30;
input_collection = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=1, age=16, gender="male", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753),
]


def MAP(_, row:NamedTuple):
    if (row.gender == 'female' and row.age > 30):
        yield (row, row)

def REDUCE_SELECT(_, rows: NamedTuple):
    yield (rows, rows)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=2, age=35, social_contacts=297, gender='female'), 297.0),
 (User(id=1, age=32, social_contacts=753, gender='female'), 753.0)]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [41]:
#Проекция по столбцу gender
input_collection = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=1, age=16, gender="male", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753),
]

def MAP(_, row:NamedTuple):
    yield (row.gender, row.gender)

def REDUCE(row, rows: NamedTuple):
    yield (row, rows[0])

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('male', 'male'), ('female', 'female')]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [42]:
input_collection_a = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

input_collection_b = [
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753)
]

def MAP(_, row:NamedTuple):
    yield (row, row)

def REDUCE(row, rows: NamedTuple):
    yield (row, rows[0])

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=2, age=54, social_contacts=120, gender='male'),
  User(id=2, age=54, social_contacts=120, gender='male')),
 (User(id=0, age=25, social_contacts=540, gender='female'),
  User(id=0, age=25, social_contacts=540, gender='female')),
 (User(id=3, age=27, social_contacts=742, gender='male'),
  User(id=3, age=27, social_contacts=742, gender='male')),
 (User(id=3, age=31, social_contacts=521, gender='male'),
  User(id=3, age=31, social_contacts=521, gender='male')),
 (User(id=2, age=35, social_contacts=297, gender='female'),
  User(id=2, age=35, social_contacts=297, gender='female')),
 (User(id=1, age=16, social_contacts=123, gender='male'),
  User(id=1, age=16, social_contacts=123, gender='male')),
 (User(id=1, age=32, social_contacts=753, gender='female'),
  User(id=1, age=32, social_contacts=753, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [43]:
input_collection_a = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

input_collection_b = [
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753)
]

def MAP(_, row:NamedTuple):
    yield (row, row)

def REDUCE(row, rows: NamedTuple):
  if len(rows) == 2:
    yield (row, rows[0])

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=3, age=31, social_contacts=521, gender='male'),
  User(id=3, age=31, social_contacts=521, gender='male')),
 (User(id=2, age=35, social_contacts=297, gender='female'),
  User(id=2, age=35, social_contacts=297, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [44]:
input_collection_a = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

input_collection_b = [
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753)
]
def MAP(_, row:NamedTuple):
  if row in input_collection_b:
    yield (row, 'S')
  if row in input_collection_a:
    yield (row, 'R')

def REDUCE(row, charic):
  if charic[0] == 'R':
    yield (row, row)

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=2, age=54, social_contacts=120, gender='male'),
  User(id=2, age=54, social_contacts=120, gender='male')),
 (User(id=0, age=25, social_contacts=540, gender='female'),
  User(id=0, age=25, social_contacts=540, gender='female')),
 (User(id=3, age=27, social_contacts=742, gender='male'),
  User(id=3, age=27, social_contacts=742, gender='male')),
 (User(id=1, age=16, social_contacts=123, gender='male'),
  User(id=1, age=16, social_contacts=123, gender='male'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [45]:
input_collection_a = [
    User(id=5, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=4, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

class User_Tel(NamedTuple):
  id: int
  tel: int
  info: str

input_collection_b = [
    User_Tel(id=2, tel=763672, info="None"),
    User_Tel(id=3, tel=899080, info="None"),
    User_Tel(id=1, tel=708090, info="None"),
    User_Tel(id=10, tel=708090, info="None"),
]

def MAP(_, row:NamedTuple):
  if row in input_collection_a and row.id in [r.id for r in input_collection_b]:
    yield (row.id, ('R', (row.age, row.gender, row.social_contacts)))
  elif row in input_collection_b and row.id in [r.id for r in input_collection_a]:
    yield (row.id, ('S', (row.tel, row.info)))


def REDUCE(row, charic):
  for i in range(len(charic)):
    for j in range(len(charic)):
      if charic[i][0] == 'R' and charic[j][0] == 'S':
        yield (None, (charic[i][1], row, charic[j][1]))

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(None, ((27, 'male', 742), 3, (899080, 'None'))),
 (None, ((35, 'female', 297), 2, (763672, 'None'))),
 (None, ((16, 'male', 123), 1, (708090, 'None')))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [46]:
input_collection_a = [
    User(id=5, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=4, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

def MAP(_, row:NamedTuple):
  yield (row.gender, row.social_contacts)

def REDUCE(row, charic): #GROUP BY USER.GENDER HAVING MAX(USER.SOCIAL_CONTACTS)
  yield (row, max(charic))

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('male', 742), ('female', 540)]

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [47]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [48]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(len(small_mat)):
        yield((i,k), w)
  # solution code that yield(k2,v2) pairs
  
def REDUCE(key, values):
    (i, k) = key
    val = 0
    for j in range(J):
        val += small_mat[i, j] * values[j]
    yield((i,k), val)
  # solution code that yield(k3,v3) pairs
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[((0, 0), 0.2651847269402926),
 ((1, 0), 0.35321751869903395),
 ((0, 1), 0.5992016587948666),
 ((1, 1), 0.9093857153126474),
 ((0, 2), 0.39185446259859197),
 ((1, 2), 0.41195074919707286),
 ((0, 3), 0.40311661316563024),
 ((1, 3), 0.7534511515357474),
 ((0, 4), 0.3978834619616908),
 ((1, 4), 0.316132707135748),
 ((0, 5), 0.5768727031888764),
 ((1, 5), 1.082407521450127),
 ((0, 6), 0.11513551520488448),
 ((1, 6), 0.6140178489645312),
 ((0, 7), 0.5595203426089735),
 ((1, 7), 0.9020896231954509),
 ((0, 8), 0.3106103470163357),
 ((1, 8), 0.1986684481250151),
 ((0, 9), 0.8553113249930392),
 ((1, 9), 0.9701216998190891),
 ((0, 10), 0.4070211235448421),
 ((1, 10), 0.5204584165615396),
 ((0, 11), 0.6536555676884832),
 ((1, 11), 0.8234258889321839),
 ((0, 12), 0.6061231764776503),
 ((1, 12), 0.679453343363557),
 ((0, 13), 0.49903670805466405),
 ((1, 13), 1.1009612337927992),
 ((0, 14), 1.1362701007718106),
 ((1, 14), 0.8956563240093172),
 ((0, 15), 0.26703596185956047),
 ((1, 15), 0.26953877408

Проверьте своё решение

In [49]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [50]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [51]:
I = 2 
J = 3
K = 4
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
print(small_mat)
print(big_mat)
reference_solution = np.matmul(small_mat, big_mat) 
print(reference_solution)
def RECORDREADER():
    for j in range(small_mat.shape[0]):
        for k in range(small_mat.shape[1]):
            yield ((0, big_mat.shape[1], small_mat.shape[1], j,k), small_mat[j,k])
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((1, small_mat.shape[0], big_mat.shape[0], j,k), big_mat[j,k])

def MAP(k1, v1):
    (n, b, a, j, k) = k1
    w = v1
    if n == 0:
        for i in range(b):
            yield((a, j, i), w)
    else:
        for i in range(b):
            yield((a, i, k), w)
    

def REDUCE(key, values):
    (a, i, k) = key
    s = 0
    for j in range(int(len(values)/2)):
        s += values[j]*values[j+a]
    yield((i,k), s)
  # solution code that yield(k3,v3) pairs
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[[0.24544599 0.43937236 0.77560741]
 [0.4124602  0.94804599 0.28026343]]
[[0.25667778 0.77839921 0.86066041 0.56518773]
 [0.11834276 0.98399998 0.50127133 0.91972493]
 [0.23990687 0.04398498 0.54700828 0.53986111]]
[[0.30107062 0.65751243 0.85575409 0.96154505]
 [0.28530086 1.2662633  0.98352285 1.2563623 ]]


[((0, 0), 0.3010706154194078),
 ((0, 1), 0.657512431577875),
 ((0, 2), 0.8557540855905859),
 ((0, 3), 0.9615450508867021),
 ((1, 0), 0.2853008633724757),
 ((1, 1), 1.2662633036533022),
 ((1, 2), 0.9835228463326523),
 ((1, 3), 1.2563622966422565)]

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [52]:
I = 2
J = 3
K = 4
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
print(small_mat)
print(big_mat)
reference_solution = np.matmul(small_mat, big_mat) 
print(reference_solution)

def INPUTFORMAT():
        
    def RECORDREADER(m1, m2, a, b):
        #print(m1,m2)
        for j in range(m1.shape[0]):
            for k in range(m1.shape[1]):
                yield ((0, a, b, m2.shape[1], m1.shape[1], j,k), m1[j,k])
        for j in range(m2.shape[0]):
            for k in range(m2.shape[1]):
                yield ((1, a, b, m1.shape[0], m2.shape[0], j,k), m2[j,k])
    
    size =  1
    for i in range(0, len(small_mat), size):
        for j in range(0, len(big_mat[0]), size):
            yield RECORDREADER(small_mat[i:i+size, :], big_mat[:, j:j+size], i, j)



def MAP(k1, v1):
    (n, a, b, A, B, j, k) = k1
    #print(a, b, A, B)
    w = v1
    if n == 0:
        for i in range(A):
            yield((B, a, b), w)
    else:
        for i in range(A):
            yield((B, a, b), w)
    

def REDUCE(key, values):
    (a, i, k) = key
    print(list(set(values)))
    s = 0
    for j in range(int(len(values)/2)):
        s += values[j]*values[j+a]
    yield((i,k), s)
  # solution code that yield(k3,v3) pairs
#output = MapReduce(RECORDREADER, MAP, REDUCE)
#output = list(output)
#output
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

[[0.28645248 0.39764924 0.79799559]
 [0.56681792 0.76683925 0.44909423]]
[[0.85154999 0.38008325 0.98223867 0.29051172]
 [0.99872709 0.03445472 0.48005651 0.87036503]
 [0.9295752  0.99791774 0.69713854 0.03251766]]
[[1.38286859 0.91891064 1.02857229 0.45526674]
 [1.66600378 0.69001832 1.23795754 0.8467008 ]]
48 key-value pairs were sent over a network.
[0.28645247823378994, 0.397649243304676, 0.7979955934419729, 0.8515499931122675, 0.9987270864466207, 0.9295751953881337]
[0.28645247823378994, 0.397649243304676, 0.7979955934419729, 0.3800832488658744, 0.03445471560546898, 0.9979177369857298]
[0.5668179151641699, 0.7668392482734478, 0.9822386680429351, 0.44909423359947764, 0.48005651078190703, 0.6971385359141465]
[0.5668179151641699, 0.7668392482734478, 0.8703650276134287, 0.44909423359947764, 0.2905117218286932, 0.03251765620711555]
[0.28645247823378994, 0.397649243304676, 0.7979955934419729, 0.9822386680429351, 0.48005651078190703, 0.6971385359141465]
[0.28645247823378994, 0.3976492433

[(0,
  [((0, 0), 1.3828685857530498),
   ((0, 1), 0.9189106368937707),
   ((1, 2), 1.237957544369525),
   ((1, 3), 0.8467008038890594)]),
 (1,
  [((0, 2), 1.0285722886119415),
   ((0, 3), 0.45526674366546716),
   ((1, 0), 1.6660037799009348),
   ((1, 1), 0.6900183242122839)])]

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [53]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица

  global maps
  split_size =  int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица

  split_size =  int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.3812178295725516), ((0, 1), 0.14522649430723028), ((0, 2), 0.17841535208294093), ((0, 3), 0.26742054423528844), ((0, 4), 0.3786466633271507), ((0, 5), 0.24013040905323302), ((0, 6), 0.13126799715635665), ((0, 7), 0.10938521652893331), ((0, 8), 0.6200811434217469), ((0, 9), 0.40659007233374733), ((0, 10), 0.15889779008049423), ((0, 11), 0.15396832457737444), ((0, 12), 0.19855771073016193), ((0, 13), 0.4495527793704409), ((0, 14), 0.06670524122243732), ((0, 15), 0.5808215601268693), ((0, 16), 0.1743052541921276), ((0, 17), 0.41238285607057257), ((0, 18), 0.6224784470851074), ((0, 19), 0.041528279919156634), ((0, 20), 0.17114120306866992), ((0, 21), 0.20406614811110108), ((0, 22), 0.5471082518676902), ((0, 23), 0.2310499969638958), ((0, 24), 0.2542986403239763), ((0, 25), 0.19537733095336787), ((0, 26), 0.39910727676369673), ((0, 27), 0.5853093627891998), ((0, 28), 0.18210096928279346), ((0, 29), 0.0743255177881592), ((0, 30),

True